In [70]:
import torch.nn as nn
import torch.nn.functional as F

class DiseaseAndTypeClassifier(nn.Module):
    def __init__(self,tmodel):
        '''
        tmodel: pretrained model
        ex:
        model_name='resnet26d'
        tmodel=timm.create_model(model_name, pretrained=True)
        m1=DiseaseAndTypeClassifier(tmodel)
        
        '''
        super().__init__()
        self.m = tmodel
        
        #do this if you want to replace head of self.m
        #get model head
        # self.
#         h=list(self.m.named_modules())[-1]
#         print(h[0])
#         h[1]=nn.Linear(in_features=self.m.get_classifier().in_features,out_features=99, bias=False)
#         print(list(self.m.named_modules())[-1])
        
        self.s=nn.Sequential(
            #do this if you want to add a head that matches the output size of self.m
            nn.Linear(in_features=self.m.get_classifier().out_features,out_features=512, bias=False),                
            nn.ReLU())
        self.l1=nn.Linear(in_features=512, out_features=10, bias=False)  #rice type
        self.l2=nn.Linear(in_features=512, out_features=10, bias=False)  #disease
        
    def forward(self,x): 
        x=self.m(x)
        x=self.s(x)     
        label=self.l1(x)  #disease type
        variety=self.l2(x)  #variety
        return label,variety
 
class smallm(nn.Module):
    def __init__(self):
        super().__init__()
        #linear network expects linear input
        self.flatten=nn.Flatten()
        self.l1=nn.Sequential(
            nn.Linear(in_features=256,out_features=512, bias=False),
            nn.ReLU())
        self.l2=nn.Sequential(
            nn.Linear(in_features=512,out_features=512, bias=False),
            nn.ReLU())
        # self.head=nn.Sequential(OrderedDict([('fc1',nn.Linear(in_features=256,out_features=512, bias=False)),
        #          ('fc2',nn.Linear(in_features=512,out_features=512, bias=False))])
        self.head=nn.Sequential(
            nn.Linear(in_features=76,out_features=99, bias=False),
            nn.ReLU())        
    def forward(self,x):
        x=self.flatten(x)
        x=self.l1(x)
        x=self.l2(x)
        x=self.head[0](x)
        x=self.head[1](x)
        return x
    
    def get_classifier(self):
        #you can see what type it is and input and output params
        return self.head[0]
        

# m_in=nn.Sequential(
#             nn.Linear(in_features=self.m.get_classifier().in_features,out_features=512, bias=False),
#             nn.ReLU())
sm=smallm()
m1=DiseaseAndTypeClassifier(sm)


smallm(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (l1): Sequential(
    (0): Linear(in_features=256, out_features=512, bias=False)
    (1): ReLU()
  )
  (l2): Sequential(
    (0): Linear(in_features=512, out_features=512, bias=False)
    (1): ReLU()
  )
  (head): Sequential(
    (0): Linear(in_features=76, out_features=99, bias=False)
    (1): ReLU()
  )
)

In [75]:
#want to remove the head?  Works with forward as well (head, last sequential is removed from it)
#but it then becomes a nn.Sequential object, you lose all the stuff that made it a smallm model
s=smallm()
s=nn.Sequential(*list(s.children())[:-1])
s

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Sequential(
    (0): Linear(in_features=256, out_features=512, bias=False)
    (1): ReLU()
  )
  (2): Sequential(
    (0): Linear(in_features=512, out_features=512, bias=False)
    (1): ReLU()
  )
)

In [82]:
import timm
tmodel=timm.create_model('resnet26d', pretrained=True, num_classes=512,global_pool='catavgmax') 
print(tmodel.get_classifier().in_features)

4096


In [83]:
#strip the head and convert to a Sequential
nm=nn.Sequential(*list(tmodel.children())[:-1])

In [84]:
nm.get_classifier().in_features

AttributeError: 'Sequential' object has no attribute 'get_classifier'

In [55]:
#get all the parameters for the l1 layer
p=list(m1.l2.parameters())

len(p[0])
p[0][9]

type(p)
p[0].requires_grad

AttributeError: 'list' object has no attribute 'l2'

In [39]:
p2=list(m1.m.head.parameters())
p2

[Parameter containing:
 tensor([[ 0.1117,  0.0419, -0.0875,  ..., -0.0762,  0.0960,  0.0056],
         [ 0.0098,  0.0295, -0.0530,  ..., -0.0715,  0.1092, -0.0794],
         [ 0.0860,  0.1053,  0.0559,  ..., -0.0113,  0.0444, -0.1116],
         ...,
         [-0.0368,  0.0990,  0.0697,  ...,  0.0029,  0.0611, -0.0217],
         [ 0.0424, -0.0100,  0.0332,  ..., -0.0335, -0.0245, -0.1022],
         [-0.0782, -0.0470,  0.0407,  ...,  0.0659, -0.0937,  0.0376]],
        requires_grad=True)]

In [43]:
for child in m1.children():
    print('FOUND1')
    print(child)

FOUND1
smallm(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (l1): Sequential(
    (0): Linear(in_features=256, out_features=512, bias=False)
    (1): ReLU()
  )
  (l2): Sequential(
    (0): Linear(in_features=512, out_features=512, bias=False)
    (1): ReLU()
  )
  (head): Sequential(
    (0): Linear(in_features=76, out_features=99, bias=False)
    (1): ReLU()
  )
)
FOUND1
Sequential(
  (0): Linear(in_features=99, out_features=512, bias=False)
  (1): ReLU()
)
FOUND1
Linear(in_features=512, out_features=10, bias=False)
FOUND1
Linear(in_features=512, out_features=10, bias=False)


In [6]:
m1.m.fc=nn.Linear(in_features=512,out_features=512)

In [7]:
print(m1)

DiseaseAndTypeClassifier(
  (m): smallm(
    (l1): Sequential(
      (0): Linear(in_features=10, out_features=512, bias=False)
      (1): ReLU()
    )
    (l12): Sequential(
      (0): Linear(in_features=512, out_features=256, bias=False)
      (1): ReLU()
    )
    (fc): Linear(in_features=512, out_features=512, bias=True)
  )
  (rl): ReLU()
  (l1): Linear(in_features=512, out_features=10, bias=False)
  (l2): Linear(in_features=512, out_features=10, bias=False)
)


In [ ]:
input_image = torch.rand(3,28,28)

In [38]:
sm=smallm()
print(sm)

smallm(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (l1): Sequential(
    (0): Linear(in_features=256, out_features=512, bias=False)
    (1): ReLU()
  )
  (l2): Sequential(
    (0): Linear(in_features=512, out_features=512, bias=False)
    (1): ReLU()
  )
  (fc): Linear(in_features=512, out_features=256, bias=False)
)
